<a href="https://colab.research.google.com/github/Facktorial/ToyDataScraping/blob/main/Orienteering/NabushenostEventu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import requests
import pandas as pd
import io


# getEventStartLists

def get_csos_club_list():
    url = "https://oris.orientacnisporty.cz/API/"
    params = {
        'format': 'json',
        'method': 'getCSOSClubList'
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Check if the request was successful
    return response.json()

def download_and_load_csv(gender: str, date: str) -> pd.DataFrame:
    url = "https://oris.orientacnisporty.cz/ranking_export"
    params = {
        'date': date,
        'sport': '1',
        'gender': gender,
        'csv': '1'
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Check if the request was successful

    csv_data = response.content.decode('utf-8')
    df = pd.read_csv(io.StringIO(csv_data), sep=";")
    return df

def download_rankings(s_time1, s_time2) -> dict[str, list[pd.DataFrame]]:
  ranking_dfs = {}
  ranking_dfs["M"] = [download_and_load_csv("M", s_time1), download_and_load_csv("M", s_time2)]
  ranking_dfs["F"] = [download_and_load_csv("F", s_time1), download_and_load_csv("F", s_time2)]
  print("Ranking Data:")
  #_ = [print(x.describe()) for x in ranking_dfs["M"]]
  #_ = [print(x.describe()) for x in ranking_dfs["F"]]
  return ranking_dfs

def get_event(event_id: int):
    url = "https://oris.orientacnisporty.cz/API/"
    params = {
      'format': 'json',
      'method': 'getEvent',
      'id': f"{event_id}"
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Check if the request was successful
    return response.json()

def get_event_entries(event_id: int=8657):
    url = "https://oris.orientacnisporty.cz/API/"
    params = {
      'format': 'json',
      'method': 'getEventEntries',
      'eventid': f"{event_id}"
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Check if the request was successful
    return response.json()

In [2]:
PAST_RANKING = tuple[float, float, float]
RANKING = tuple[float, float, float, PAST_RANKING]


def get_ranking_by_regno(df: pd.DataFrame, minule_df: pd.DataFrame, reg_no: str) -> RANKING:
  filtered_df = df[df['Reg. c.'] == reg_no]
  minule_filtered_df = minule_df[minule_df['Reg. c.'] == reg_no]
  if filtered_df.empty:
    return (float('inf'), -1, -1, (float('inf'), float('inf'), float('inf')))
  if minule_filtered_df.empty:
    minule = filtered_df['Minule Ran'].iloc[0]
    return (
      filtered_df['Poradi'].iloc[0]
      , filtered_df['Body'].iloc[0]
      , filtered_df['Rank.c'].iloc[0]
      , (
          minule if minule > 0 else float('inf')
          , float('inf')
          , float('inf')
      )
    )
  return (
    filtered_df['Poradi'].iloc[0]
    , filtered_df['Body'].iloc[0]
    , filtered_df['Rank.c'].iloc[0]
    , (
        filtered_df['Minule Ran'].iloc[0]
        , minule_filtered_df['Poradi'].iloc[0]
        , minule_filtered_df['Minule Ran'].iloc[0]
    )
  )

In [39]:
from enum import Enum


class MultiRaceClass(Enum):
  H21E = "H21E"
  D21E = "D21E"
  H21A = "H21A"
  D21A = "D21A"
  H21B = "H21B"
  D21B = "D21B"
  H21C = "H21C"
  D21C = "D21C"
  H_OTHER = "H_OTHER"
  D_OTHER = "D_OTHER"
  OTHER = ""

class AcademicRaceClass(Enum):
  H = "H"
  D = "D"
  H_OTHER = "H_OTHER"
  D_OTHER = "D_OTHER"
  OTHER = ""

def class_2_gender(r: Enum) -> str:
  if r.value == "":
    return ""
  return "M" if r.value[0] == "H" else "F" if r.value[0] == "D" else ""

# get_gender_akademicky = get_gender[AcademicRaceClass]

def get_gender_multi(s: str) -> MultiRaceClass:
  for e in MultiRaceClass:
    if e.value == s:
      return e
  if "H" in s:
    return MultiRaceClass.H_OTHER
  elif "D" in s:
    return MultiRaceClass.D_OTHER
  return MultiRaceClass.OTHER

def get_gender_akademicky(s: str) -> AcademicRaceClass:
  if AcademicRaceClass.H.value == s:
    return AcademicRaceClass.H
  elif AcademicRaceClass.D.value == s:
    return AcademicRaceClass.D
  elif "H" in s:
    return AcademicRaceClass.H_OTHER
  elif "D" in s:
    return AcademicRaceClass.D_OTHER
  return AcademicRaceClass.OTHER

In [49]:
from dataclasses import dataclass
from typing import TypeVar, Type, Union


T = TypeVar('T', bound='Entry')
EntryUnion = Union['AcademicRaceClass', 'MultiRaceClass']
EntryTypes = Type[EntryUnion]

@dataclass
class Entry:
  classR: EntryTypes
  name: str
  regno: int
  licence: str
  ranking: RANKING

  @property
  def ranked(self) -> float:
    if self.ranking is None:
      return None
    return self.ranking[0]


@dataclass
class MultiEntry(Entry):
  classR: MultiRaceClass

  @staticmethod
  def get_gender(class_desc: str) -> 'MultiRaceClass':
    return get_gender_multi(class_desc)


@dataclass
class AcademicEntry(Entry):
  classR: AcademicRaceClass

  @staticmethod
  def get_gender(class_desc: str) -> 'AcademicRaceClass':
    return get_gender_akademicky(class_desc)


def divide_entries(entries: list[T], race_enum: EntryTypes) -> dict[EntryUnion, list[T]]:
  ret = {race_class: [] for race_class in race_enum}
  for entry in entries:
    ret[entry.classR].append(entry)
  for cls, ls in ret.items():
    ret[cls] = sorted(ls, key=lambda entry: (entry.ranked is None, entry.ranked))
    #ret[cls] = sorted(ls, key=lambda entry: entry.ranked)
  return ret

def mk_entries(entries: pd.DataFrame, ranking_dfs: dict[str, list[pd.DataFrame]], entry_type: Type[T]) -> list[T]:
  entrs: list[T] = []
  for x in entries["Data"]:
    obj = entries["Data"][x]
    cls = entry_type.get_gender(obj["ClassDesc"])
    gender = class_2_gender(cls)
    # print(gender, entry_type)
    entrs.append(Entry(
        classR=cls
        , name=obj["Name"]
        , regno=obj["RegNo"]
        , licence=obj['Licence']
        , ranking=get_ranking_by_regno(ranking_dfs[gender][0], ranking_dfs[gender][1], obj["RegNo"])
        if gender != "" and cls != entry_type.__annotations__.get('classR').OTHER else None
    ))
  return entrs

In [38]:
def print_analyze(div_entrs: dict[Enum, list[Entry]], event_id: int) -> None:
  print(f"RANKING z ORISU, event #{event_id}")
  for r, ls in div_entrs.items():
    if not len(ls):
      continue
    print(f"{r} ({len(ls)})")
    for id, x in enumerate(ls):
      print(f"{id}. ({x.licence}) {x.name.ljust(20)} \t{x.ranked}.\t{x.ranking[3] if x.ranking is not None else ''}.")

In [6]:
ranking_dfs = download_rankings('2024-05-31', '2024-03-31')

Ranking Data:


In [25]:
from functools import partial

class PrintRes(Enum):
  YES = True
  NO = False

def analyze_event_inner(is_print: PrintRes, event_id: int, ranking_dfs, entry_type: Entry, gender=None, age=None, level=None) -> None | dict[EntryUnion, any]: # type hint
  entries = get_event_entries(event_id)

  enum_type = entry_type.__annotations__.get('classR')

  entrs = mk_entries(entries, ranking_dfs, entry_type)
  div_entrs = divide_entries(entrs, enum_type)

  div_entrs = filter_entries(enum_type, div_entrs, gender, age, level)

  if (is_print == PrintRes.YES):
    print_analyze(div_entrs, event_id)
    return

  return div_entrs

analyze_event = partial(analyze_event_inner, PrintRes.YES)
get_analyze = partial(analyze_event_inner, PrintRes.NO)

def filter_entries(enum, entries, gender, age, level) -> dict[EntryUnion, any]:
  categories = get_x_y_categories(enum, gender, age, level)
  return { k: v for k, v in entries.items() if k in categories }

def category_cond(gender, age, level, enum_instance) -> bool:
  return all([(x is None or x in enum_instance.value) for x in [gender, None if age is None else str(age), level]])

def get_x_y_categories(enum, gender, age, level) -> dict[EntryUnion, any]:
  cond = partial(category_cond, gender, age, level)
  return [e for e in enum if cond(e)]

In [ ]:
analyze_event(8124, ranking_dfs, MultiEntry, "H", 21, None)

In [ ]:
analyze_event(8657, ranking_dfs, AcademicEntry, None, None, None)

In [ ]:
analyze_event(7609, ranking_dfs, MultiEntry, "H", 21, None)

In [ ]:
analyze_event(7906, ranking_dfs, MultiEntry, "H", 21, None)

In [51]:
analyze_event(7744, ranking_dfs, MultiEntry, "H", 21, None)

RANKING z ORISU, event #7744
MultiRaceClass.H21B (30)
0. (R) Poklop Martin        	22.	(25, 28, 27).
1. (E) Kozák Osvald         	26.	(17, 17, 17).
2. (R) Procházka David      	28.	(24, 25, 29).
3. (C) Smola Michal         	92.	(107, 106, 108).
4. (C) Jílek Jakub          	111.	(135, 152, 152).
5. (A) Sláma Šimon          	117.	(160, 189, 299).
6. (C) Rychlý Pavel         	121.	(105, 104, 106).
7. (C) Zelinka Jiří         	131.	(127, 147, 144).
8. (C) Mokrý Ondřej         	135.	(125, 128, 125).
9. (C) Štorek David         	147.	(197, 208, 250).
10. (A) Štorek Jan           	174.	(422, 668, 615).
11. (C) Zimmermann Jakub     	181.	(174, 196, 200).
12. (C) Rajnošek Zdeněk      	193.	(177, 240, 239).
13. (C) Drábek Jan           	218.	(225, 219, 214).
14. (C) Bukovac Maroš        	249.	(253, 247, 244).
15. (C) Bjolek Michal        	287.	(507, 527, 518).
16. (C) Skřivanek Marcel     	308.	(309, 311, 311).
17. (C) Vavřík Tomáš         	327.	(319, 309, 309).
18. (C) Seidenglanz Jáchym   	377

In [50]:
analyze_event(7745, ranking_dfs, MultiEntry, None, 21, None)

RANKING z ORISU, event #7745
MultiRaceClass.H21B (43)
0. (R) Poklop Martin        	22.	(25, 28, 27).
1. (E) Kozák Osvald         	26.	(17, 17, 17).
2. (C) Hovorka Lukáš        	63.	(74, 82, 83).
3. (C) Smola Michal         	92.	(107, 106, 108).
4. (A) Sláma Šimon          	117.	(160, 189, 299).
5. (C) Rychlý Pavel         	121.	(105, 104, 106).
6. (C) Zelinka Jiří         	131.	(127, 147, 144).
7. (C) Štorek David         	147.	(197, 208, 250).
8. (C) Goldschmidt Tobiáš   	160.	(153, 142, 139).
9. (A) Štorek Jan           	174.	(422, 668, 615).
10. (C) Otrusina Jiří        	187.	(178, 179, 172).
11. (C) Rajnošek Zdeněk      	193.	(177, 240, 239).
12. (C) Ehl Jiří             	199.	(193, 188, 186).
13. (C) Zimmermann Štěpán    	215.	(210, 297, 292).
14. (C) Drábek Jan           	218.	(225, 219, 214).
15. (C) Bukovac Maroš        	249.	(253, 247, 244).
16. (C) Perknovský Radim     	267.	(338, 332, 328).
17. (C) Bencúr Juraj         	280.	(272, 267, 258).
18. (C) Bjolek Michal        	287

In [76]:
def destruct_category(cat) -> tuple[str, int, str]:
  val = cat.value
  g = val[0]
  age = int(val[1] + val[2]) if len(val) > 2 else None
  lvl = val[3] if len(val) > 3 else None
  return (g, age, lvl)

def model_of_vyteznost(events: list[int], ranking_dfs, type_entry, category):
  mock_standing = 1
  for event in events:
    print(f"\nMODELING EVENT #{event}")
    anal = get_analyze(event, ranking_dfs, type_entry, *destruct_category(category))
    coef = float(get_event(event)["Data"]["RankingKoef"].replace(',','.'))

    points_basic = sum([x.ranking[2] for x in anal[category][0:5]])/5
    count_racers = len(anal[category])

    _ = [print(x.name, x.ranking[2]) for x in anal[category][0:5]]
    print("=====", points_basic)

    interval_race = partial(points_i, 0.0)
    specific_race = partial(interval_race, coef, points_basic, count_racers)

    model_win_times: list[int] = [x*60 for x in range(30, 48)] # FIXME!!!!

    for x in model_win_times:
      model_time: int = x
      print(f"\nMODEL TIME: {model_time/60}min:")
      case_study = partial(specific_race, model_time)
      model_times = [x for x in range(model_time, 2*model_time, 60)]
      _ = [print(f"{x//60}min:\t", case_study(x, mock_standing), "pts") for x in model_times]

model_of_vyteznost([7744, 7745], ranking_dfs, MultiEntry, MultiRaceClass.H21B)


MODELING EVENT #7744
Poklop Martin 8692
Kozák Osvald 8577
Procházka David 8520
Smola Michal 7769
Jílek Jakub 7657
===== 8243.0

MODEL TIME: 30.0min:
30min:	 8572 pts
31min:	 8286 pts
32min:	 8001 pts
33min:	 7715 pts
34min:	 7429 pts
35min:	 7143 pts
36min:	 6858 pts
37min:	 6572 pts
38min:	 6286 pts
39min:	 6000 pts
40min:	 5715 pts
41min:	 5429 pts
42min:	 5143 pts
43min:	 4857 pts
44min:	 4572 pts
45min:	 4286 pts
46min:	 4000 pts
47min:	 3714 pts
48min:	 3429 pts
49min:	 3143 pts
50min:	 2857 pts
51min:	 2571 pts
52min:	 2286 pts
53min:	 2000 pts
54min:	 1714 pts
55min:	 1428 pts
56min:	 1143 pts
57min:	 857 pts
58min:	 571 pts
59min:	 285 pts

MODEL TIME: 31.0min:
31min:	 8572 pts
32min:	 8296 pts
33min:	 8019 pts
34min:	 7743 pts
35min:	 7466 pts
36min:	 7190 pts
37min:	 6913 pts
38min:	 6636 pts
39min:	 6360 pts
40min:	 6083 pts
41min:	 5807 pts
42min:	 5530 pts
43min:	 5254 pts
44min:	 4977 pts
45min:	 4701 pts
46min:	 4424 pts
47min:	 4148 pts
48min:	 3871 pts
49min:	 3595 pt

In [14]:
def points_i(race_factor: float, race_coef: float, race_points: float, n_racers: int, t_s: int, t_i: int, standing_i: int) -> int:
  return int((2 - t_i/t_s) * race_points * (1 - race_factor * (standing_i - 1) / (n_racers - 1)) * race_coef)


category = AcademicRaceClass.H
anal = get_analyze(8657, ranking_dfs, AcademicEntry, None, None, None)
mock_standing = 1
coef = 1.00

points_basic = sum([x.ranking[2] for x in anal[category][0:5]])/5
count_racers = len(anal[category])

_ = [print(x.name, x.ranking[2]) for x in anal[category][0:5]]
print("=====", points_basic)

interval_race = partial(points_i, 0.0)
specific_race = partial(interval_race, coef, points_basic, count_racers)

model_win_times: list[int] = [x*60 for x in range(10, 15)]

for x in model_win_times:
  model_time: int = x
  print(f"\nMODEL TIME: {model_time/60}min:")
  case_study = partial(specific_race, model_time)
  model_times = [x for x in range(model_time, 2*model_time, 60)]
  _ = [print(f"{x//60}min:\t", case_study(x, mock_standing), "pts") for x in model_times]

Vandas Daniel 9500
Roudný Martin 9466
Coufal Jáchym 9115
Janovský Tomáš 9007
Horvát Petr 8966
===== 9210.8

MODEL TIME: 10.0min:
10min:	 9210 pts
11min:	 8289 pts
12min:	 7368 pts
13min:	 6447 pts
14min:	 5526 pts
15min:	 4605 pts
16min:	 3684 pts
17min:	 2763 pts
18min:	 1842 pts
19min:	 921 pts

MODEL TIME: 11.0min:
11min:	 9210 pts
12min:	 8373 pts
13min:	 7536 pts
14min:	 6698 pts
15min:	 5861 pts
16min:	 5024 pts
17min:	 4186 pts
18min:	 3349 pts
19min:	 2512 pts
20min:	 1674 pts
21min:	 837 pts

MODEL TIME: 12.0min:
12min:	 9210 pts
13min:	 8443 pts
14min:	 7675 pts
15min:	 6908 pts
16min:	 6140 pts
17min:	 5372 pts
18min:	 4605 pts
19min:	 3837 pts
20min:	 3070 pts
21min:	 2302 pts
22min:	 1535 pts
23min:	 767 pts

MODEL TIME: 13.0min:
13min:	 9210 pts
14min:	 8502 pts
15min:	 7793 pts
16min:	 7085 pts
17min:	 6376 pts
18min:	 5668 pts
19min:	 4959 pts
20min:	 4251 pts
21min:	 3542 pts
22min:	 2834 pts
23min:	 2125 pts
24min:	 1417 pts
25min:	 708 pts

MODEL TIME: 14.0min:
14min